In [1]:
from data.data import COVID19India

In [2]:
cin = COVID19India()

In [3]:
df = cin.moh_data(save=False)

In [4]:
df.head()

,Name of State / UT,Total Confirmed cases (Indian National),Total Confirmed cases ( Foreign National ),Cured/Discharged/Migrated,Death
0,Andhra Pradesh,3,0,0,0
1,Chhattisgarh,1,0,0,0
2,Delhi,26,1,5,1
3,Gujarat,14,0,0,0
4,Haryana,3,14,0,0


df

In [5]:
df.head()

,Name of State / UT,Total Confirmed cases (Indian National),Total Confirmed cases ( Foreign National ),Cured/Discharged/Migrated,Death
0,Andhra Pradesh,3,0,0,0
1,Chhattisgarh,1,0,0,0
2,Delhi,26,1,5,1
3,Gujarat,14,0,0,0
4,Haryana,3,14,0,0


In [18]:
from pathlib import Path

In [ ]:
[349, 41, 24, 7]

In [3]:
349+41+7

397